In [ ]:
"""
Created on Mon Dec 12 11:28:37 2022
This script formats the US County shapefile downloaded from the US Census Bureau 
to create a csv containing county-level geographic area data for use in 
downselecting ReEDS BA data to county-level resolution. The output csv file
created by this script is in the same format as the population downselection 
csv (co-est2021-alldata.csv)
@author: jcarag
"""
import os
import pandas as pd
import geopandas as gpd
import numpy as np

reedsdir = os.path.join('D:\\','akash','reeds','ReEDS-2.0')
projectdir = os.path.join(os.path.dirname(os.getcwd()))
output_dir = os.getcwd()

#######################
#%% Helpful dictionaries
#######################
fipsremove = {
    '02' : 'Alaska',
    '15' : 'Hawaii',
    '60' : 'American Samoa',
    '66' : 'Guam',
    '69' : 'Northern Mariana Islands',
    '72' : 'Puerto Rico',
    '78' : 'US Virgin Islands',
    }
countyfix = {}

#######################
#%% Import shapefile, downselect to conterminous US
#######################

shpfileloc = os.path.join(projectdir,'processing_scripts','shapefiles','tl_2021_us_county')

crs_na = 4269

# Open the ReEDS BA shapefile
# The land area is in m^2
df_county = gpd.read_file(os.path.join(shpfileloc,'tl_2021_us_county.shp'))
df_county.to_crs(epsg=crs_na,inplace=True)
# Downselect to only conterminous US States
df_county = df_county.loc[(~df_county['STATEFP'].isin(list(fipsremove.keys())))]
# Plot geodataframe
df_county.to_crs(epsg=crs_na).plot()

#######################
#%% Format and downselect columns
#######################
df_us = df_county.copy()
# Format state, county, and FIPS coulumns as integers for uniformity
df_us.rename(columns={'GEOID':'FIPS'}, inplace=True)
df_us.FIPS = df_us.FIPS.astype(int)
df_us.STATEFP = df_us.STATEFP.astype(int)
df_us.COUNTYFP = df_us.COUNTYFP.astype(int)
# Downselect to only the necessary columns and rename to match population mapping columns
df_us_small = df_us.copy()
df_us_small = df_us_small[['STATEFP','COUNTYFP','FIPS','NAMELSAD','ALAND','AWATER']]
df_us_small.columns = ['STATE','COUNTY','FIPS','CTYNAME','ALAND','AWATER']

#######################
#%% Data adjustments to match Akash's FIPS map
#######################
df_us_small.replace({'Oglala Lakota':'Shannon', 46102:46113,
                      },
                    inplace=True)

#######################
#%% Merge to add state names to df and to compare with Akash's BA2FIPS map 
#######################
bafipsmap = pd.read_excel('county_reeds_corrected0310.xlsx',header=0,usecols=['NAME','STATE_NAME','FIPS','PCA_REG'])

df_merge = df_us_small.copy()
df_merge = df_merge.merge(bafipsmap, how='left', on='FIPS')
df_missing = df_merge.loc[df_merge['NAME'].isna()]

df_merge = df_merge[['PCA_REG','STATE','COUNTY','FIPS','STATE_NAME','CTYNAME','ALAND','AWATER']]

#######################
#%% Output version of df_merge that matches the format of population mapping file
#######################
df_output = df_merge.copy()
# Filter to only include columns consistent with population mapping file
df_output = df_output[['STATE','COUNTY','STATE_NAME','CTYNAME','ALAND','AWATER']]
df_output.rename(columns={'STATE_NAME':'STNAME'},inplace=True)
df_output.sort_values(by=['STATE','COUNTY'],inplace=True)
# Output geoarea dataframe
# The land area is in m^2


In [ ]:
df_output.to_csv(os.path.join(output_dir,'geoareamap.csv'),
                 header=True,index=False)